In [1]:
!git clone https://github.com/stefan-it/nmt-mk-en.git

Cloning into 'nmt-mk-en'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [2]:
%cd nmt-kn-ml/

/content/nmt-kn-ml


In [3]:
! chmod a+x /content/nmt-kn-ml/scripts/data_preparation.sh
! chmod a+x /content/nmt-kn-ml/scripts/split_dataset.sh

In [4]:
! /content/nmt-kn-ml/scripts/data_preparation.sh data.kn data.ml

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 148090 (delta 319), reused 443 (delta 292), pack-reused 147572
Receiving objects: 100% (148090/148090), 129.87 MiB | 19.59 MiB/s, done.
Resolving deltas: 100% (114345/114345), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 587 (delta 1), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (587/587), 244.20 KiB | 3.30 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 10.14 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Tokenizer Version 1.1
Language: sk
Number of threads

In [5]:
! /content/nmt-kn-ml/scripts/split_dataset.sh corpus.clean.bpe.32000.kn corpus.clean.bpe.32000.ml

In [6]:
%cd /content/

/content


In [7]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [8]:
!fairseq-preprocess --source-lang kn --target-lang ml \
--trainpref nmt-kn-ml/train/train \
--validpref nmt-kn-ml/dev/dev \
--testpref nmt-kn-ml/test/test \
--destdir nmt-kn-ml/tokenized.kn-ml \
--thresholdsrc 2 \
--thresholdtgt 2 

2022-03-05 06:52:27 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-05 06:52:28 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='nmt-kn-ml/tokenized.kn-ml', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_co

In [9]:
import wandb
wandb.login()
# login authorization.

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
!fairseq-train /content/nmt-kn-ml/tokenized.kn-ml \
--arch lstm \
  --dropout 0.2 \
  --optimizer adam --lr 0.005 --lr-shrink 0.5 \
  --max-tokens 12000 \
  --max-epoch 25 \
  --wandb-project "Simple LSTM - kn to ml - 5 March 2022"

2022-03-05 06:54:22 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-03-05 06:54:24 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-05 06:54:26 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'Simple LSTM - kn to ml - 5 March 2022', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1,

In [11]:
!pwd

/content


In [12]:
!fairseq-generate /content/nmt-kn-ml/tokenized.kn-ml \
  --path checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe

Streaming output truncated to the last 5000 lines.
D-779	-0.6462001800537109	ഇതിന ് റെ പ ് രവര ് ‍ ത ് തനങ ് ങള ് ‍ പുരോഗമിക ് കുകയാണ ് .
P-779	-2.4765 -0.0006 -0.5664 -3.2285 -0.0003 -2.5209 -0.0005 -0.0046 -0.0127 -0.0004 -0.4817 -0.0001 -0.5054 -0.0002 -0.0060 -3.3924 -0.0002 -0.2961 -0.0001 -0.0746 -0.0022
S-729	ನಾವೆಲ ್ ಲಾ ಸಮಾಜವಾದಿ ಕುಟುಂಬದ ಸದಸ ್ ಯರು .
T-729	നാം എല ് ലാവരും കുടുംബത ് തിന ് ‍ റെ ഭാഗമാണ ് .
H-729	-0.798639714717865	ഞങ ് ങള ് ‍ ഞങ ് ങള ് ‍ തമ ് മില ് ‍ പങ ് കെടുത ് തു .
D-729	-0.798639714717865	ഞങ ് ങള ് ‍ ഞങ ് ങള ് ‍ തമ ് മില ് ‍ പങ ് കെടുത ് തു .
P-729	-1.8704 -0.0009 -1.6567 -0.0016 -0.0020 -3.8966 -0.0017 -0.8559 -0.0003 -0.0024 -4.1688 -0.0036 -0.7574 -0.0001 -0.0023 -3.1613 -0.0002 -0.6716 -0.0003 -0.4905 -0.0229 -0.0025
S-104	ಅಂದರೆ , ಈವರೆಗೆ 2 ಲಕ ್ ಷ ರೂ .
T-104	നിലവില ് ‍ 2 ലക ് ഷമാണ ് നികുതിയിളവ ് .
H-104	-1.1345914602279663	2 ലക ് ഷം രൂപ അനുവദിച ് ചു .
D-104	-1.1345914602279663	2 ലക ് ഷം രൂപ അനുവദിച ് ചു .
P-104	-3.5860 -0.9494 -0.0001 -0.0839 -1.3257 -3.5831 -